In [1]:
!pip install autots
# Read in data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import time
import datetime
from sklearn.model_selection import train_test_split
from autots import AutoTS

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Import data

In [3]:
#df

In [4]:
# Define years and months to load data from both 2023 and 2024
years = [2023, 2024]
months = range(1, 13)

# Load datasets dynamically for both years
data_files = [
    f"ig_data-{year}/ig_data_{str(month).zfill(2)}-{year}_u.csv"
    for year in years
    for month in months
]

datasets = [pd.read_csv(file) for file in data_files]

# Add month and year columns
for i, (year, month) in enumerate([(y, m) for y in years for m in months]):
    datasets[i]["month"] = month
    datasets[i]["year"] = year

In [5]:
# Concatenate data
data = pd.concat(datasets)
data

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year
0,2023-01-01,26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...,20558,0,0,0,0,0,0,1,2023
1,2023-01-01,31832168975075c19bfca97103d24b2525b75235b2e61a...,5256,0,0,0,0,0,0,1,2023
2,2023-01-01,065c756ab25ca5147325477d859c320577aa171e55d99b...,337483,1,149,145,4,4101,4511,1,2023
3,2023-01-01,ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...,62556,0,0,0,0,0,0,1,2023
4,2023-01-01,27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...,359455,1,3997,3509,488,57388,63127,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
547988,2024-12-31,8c1ac76cb45374c9710165d154d153ddf48bc30c253c8e...,5905,0,0,0,0,0,0,12,2024
547989,2024-12-31,834715aec99b1e9be4feccadeea4712f1cde42d649664d...,211245,1,345,337,8,10157,11172,12,2024
547990,2024-12-31,94528532ec6bee015266fdaeb9cd570a6fbf6e3a2dfbc2...,1660,0,0,0,0,0,0,12,2024
547991,2024-12-31,0597c04c318d668379c1d549c31931c56838f6865ae7f3...,10084,1,28,28,0,2914,3205,12,2024


In [6]:
# find in the column followers the rows that have the value as "undefined"
data[data["followers"] == "undefined"]
# replace the "undefined" in follwoers with nan
data["followers_num"] = data["followers"].replace("undefined", np.nan)
# convert the followers_num to integer
data["followers_num"] = data["followers_num"].astype(float).astype(pd.Int64Dtype())
# Ensure followers_num is numeric and clean
data["followers_num"] = pd.to_numeric(data["followers_num"], errors="coerce")

# Step 4: Replace zeros with NaN (to avoid filling with zero)
data["followers_num"].replace(0, np.nan, inplace=True)

# Step 5: Fill NaN values using the last non-null value, making sure it isn't zero
data["followers_num"] = data["followers_num"].fillna(method="ffill")

# Step 6: If still NaN (if there were no valid previous non-null value), you can set to zero or some other value
data["followers_num"].fillna(0, inplace=True)

# Step 7: Ensure the column type is integer, but with nullable integer type to handle missing values
data["followers_num"] = data["followers_num"].astype(pd.Int64Dtype())

# Verify the changes
print(data.head())


         date                                         profile_id followers  \
0  2023-01-01  26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...     20558   
1  2023-01-01  31832168975075c19bfca97103d24b2525b75235b2e61a...      5256   
2  2023-01-01  065c756ab25ca5147325477d859c320577aa171e55d99b...    337483   
3  2023-01-01  ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...     62556   
4  2023-01-01  27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...    359455   

   posts  engagement  likes  comments  reach  impressions  month  year  \
0      0           0      0         0      0            0      1  2023   
1      0           0      0         0      0            0      1  2023   
2      1         149    145         4   4101         4511      1  2023   
3      0           0      0         0      0            0      1  2023   
4      1        3997   3509       488  57388        63127      1  2023   

   followers_num  
0          20558  
1           5256  
2         337483  
3         

In [7]:
data.isnull().sum()

date             0
profile_id       0
followers        0
posts            0
engagement       0
likes            0
comments         0
reach            0
impressions      0
month            0
year             0
followers_num    0
dtype: int64

In [8]:
data[data["followers_num"] == "undefined"]

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year,followers_num


In [9]:
profile_counts = data["profile_id"].value_counts()
# # get the ids of the accounts that have more than 365 records
# more_than_299_ids = profile_counts[profile_counts >= 300].index
# # filter the data to include only the accounts that have more than 365 records
# more_than_299_data = data[data["profile_id"].isin(more_than_299_ids)]
# get the ids of the accounts that have more than 365 records
more_than_365_ids = profile_counts[profile_counts > 365].index
# filter the data to include only the accounts that have more than 365 records
more_than_365_data = data[data["profile_id"].isin(more_than_365_ids)]
# get the ids of the accounts that have 300 to 365 records
between_365_300_ids = profile_counts[
    (profile_counts <= 365) & (profile_counts >= 300)
].index
# filter the data to include only the accounts that have 300 to 365 records
between_365_300_data = data[data["profile_id"].isin(between_365_300_ids)]
# get the ids of the accounts that have 90 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 90 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [12]:
# Export more than 365 data to csv
between_365_300_data.to_csv("between_365_300_data.csv", index=False)

# Export more than 365 data to csv
between_299_90_data.to_csv("between_299_90_data.csv", index=False)

# Weekly
## 365+

In [61]:
# Convert date column to datetime
more_than_365_data["date"] = pd.to_datetime(more_than_365_data["date"])

# Extract year and week number
more_than_365_data["year"] = more_than_365_data["date"].dt.year
more_than_365_data["week"] = more_than_365_data["date"].dt.isocalendar().week  # ISO week number

# Aggregate: Keep last follower count per week for each profile
weekly_df = (
    more_than_365_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "week"])
    .last()
    .reset_index()
)

# Convert back to a proper date format (set to Monday of the given week)
weekly_df["date"] = pd.to_datetime(weekly_df["year"].astype(str) + weekly_df["week"].astype(str) + '1', format='%G%V%w')

# Drop unnecessary columns
weekly_df = weekly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Extract unique IDs from the DataFrame
unique_ids = weekly_df['profile_id'].unique()

# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
weekly_df = weekly_df[weekly_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
print(f"Number of unique IDs in filtered DataFrame: {weekly_df['profile_id'].nunique()}")

weekly_df


Number of unique IDs in filtered DataFrame: 60


,date,profile_id,followers_num
28140,2023-02-13,02170cc6ad8d4839d4b103387977def165bea9631fd28d...,1239
28141,2023-02-20,02170cc6ad8d4839d4b103387977def165bea9631fd28d...,4043461
28142,2023-02-27,02170cc6ad8d4839d4b103387977def165bea9631fd28d...,4043256
28143,2023-03-06,02170cc6ad8d4839d4b103387977def165bea9631fd28d...,4042223
28144,2023-03-13,02170cc6ad8d4839d4b103387977def165bea9631fd28d...,103218
...,...,...,...
3542705,2024-06-17,ff30e89453543049bbac3f7e1ae07987f2163893b2c3c4...,35057
3542706,2024-06-24,ff30e89453543049bbac3f7e1ae07987f2163893b2c3c4...,207105
3542707,2024-07-01,ff30e89453543049bbac3f7e1ae07987f2163893b2c3c4...,229471
3542708,2024-07-08,ff30e89453543049bbac3f7e1ae07987f2163893b2c3c4...,74189


In [62]:
weekly_df.fillna(method='ffill', inplace=True)
weekly_df.fillna(method='bfill', inplace=True)
weekly_df.fillna(0, inplace=True)
# Split the data into training and testing sets
train_df = weekly_df[weekly_df['date'] < '2024-07-01']
test_df = weekly_df[weekly_df['date'] >= '2024-07-01']

# Pivot and prepare for AutoTS
def prepare_for_autots(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')  # Forward-fill missing values
    df.index = df.index.to_period('W').to_timestamp('W')  # Convert index to week-end format
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autots(train_df)
test_df = prepare_for_autots(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)


In [63]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from autots import AutoTS

# Store evaluation results
evaluation_results_w = []

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='W',  # Changed frequency to weekly
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_w.append({'Profile': profile_id, 'MAE_AutoTS_w_365': mae_autots, 'MSE_AutoTS_w_365': mse_autots})

# Convert to DataFrame
evaluation_df_w = pd.DataFrame(evaluation_results_w)

Using 4 cpus for n_jobs.
Data frequency is: W-SUN, used frequency is: W
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py

Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Model Number: 11 with model GLM in generation 0 of 10
Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number: 21 with model VAR in generation 0 of 10
Template Eval Error: ValueError('maxlags is too large for the number of observations and the number of equations. The largest model cannot be estimated.') in model 21 in generation 0: VAR
Model Number: 22 with model VAR in generation 0 of 10
Template Eval Error: ValueError('maxlags is too large for the number of observations and t

...

194 - BasicLinearModel with avg smape 140.1: 
Model Number: 195 of 191 with model BasicLinearModel for Validation 1
195 - BasicLinearModel with avg smape 134.15: 
Model Number: 196 of 191 with model FFT for Validation 1
196 - FFT with avg smape 134.55: 
Model Number: 197 of 191 with model BasicLinearModel for Validation 1
197 - BasicLinearModel with avg smape 136.08: 
Model Number: 198 of 191 with model MetricMotif for Validation 1
Template Eval Error: ValueError('kth(=14) out of bounds (7)') in model 198 in generation 0: MetricMotif
Model Number: 199 of 191 with model FFT for Validation 1
199 - FFT with avg smape 132.26: 
Model Number: 200 of 191 with model BasicLinearModel for Validation 1
200 - BasicLinearModel with avg smape 136.77: 
Model Number: 201 of 191 with model FFT for Validation 1
201 - FFT with avg smape 136.45: 
Model Number: 202 of 191 with model BasicLinearModel for Validation 1
202 - BasicLinearModel with avg smape 142.48: 
Model Number: 203 of 191 with model BasicLin

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Model Number: 1633 with model Ensemble in generation 12 of Ensembles
Model Number: 1647 with model Ensemble in generation 12 of Ensembles
Model Number: 1661 with model Ensemble in generation 12 of Ensembles
Model Number: 1675 with model Ensemble in generation 12 of Ensembles
Model Number: 1689 with model Ensemble in generation 12 of Ensembles
Model Number: 1703 with model Ensemble in generation 12 of Ensembles
Model Number: 1717 with model Ensemble in generation 12 of Ensembles
Validation Round: 1
Model Number: 1 of 16 with model Ensemble for Validation 1
prediction too long for indepedent=False, falling back on indepedent=True
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1430, in model_forecast
    df_forecast = model_forecast(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1512, in model_forecast
    raise ValueError(

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


27 - Ensemble with avg smape 117.97: 
28 - Ensemble with avg smape 117.97: 
29 - Ensemble with avg smape 0.0: 
30 - Ensemble with avg smape 131.51: 
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error IndexError('single positional indexer is out-of-bounds')") in model 30 in generation 0: Ensemble
Model Number: 31 of 16 with model Ensemble for Validation 1
31 - Ensemble with avg smape 119.36: 
32 - Ensemble with avg smape 119.33: 
33 - Ensemble with avg smape 0.0: 
34 - Ensemble with avg smape 131.56: 
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'meth

In [64]:
evaluation_results_w

[{'Profile': '02170cc6ad8d4839d4b103387977def165bea9631fd28de5379a59339bc6b942',
  'MAE_AutoTS_w_365': 25146.949999999997,
  'MSE_AutoTS_w_365': 632369094.3025},
 {'Profile': '0225ee6b651c06a7b02cf995cb466c63dc02b861b04fc5294457e5cd17c67267',
  'MAE_AutoTS_w_365': 7480.900000000001,
  'MSE_AutoTS_w_365': 55963864.81000002},
 {'Profile': '077e1d3b46ac249ee51fdb604c82dfbdf350f46a48fe2ca20ea3bc11614cec64',
  'MAE_AutoTS_w_365': 245818.30769230778,
  'MSE_AutoTS_w_365': 60451406331.76925},
 {'Profile': '0c997f518f0769c39126acb6538cae0f9c945346abecc99705370a49c377467e',
  'MAE_AutoTS_w_365': 75094.75,
  'MSE_AutoTS_w_365': 5639221477.5625},
 {'Profile': '100b80f30629b949b2b3015e944f81dbdee219520fecab758468d3e0c433e8f1',
  'MAE_AutoTS_w_365': 172708.2653846154,
  'MSE_AutoTS_w_365': 387728760899.6302},
 {'Profile': '10bcfb1593e1c348b04c1aa2aeb4d1add22a9ffe71ba7340e9649574b01b1dbe',
  'MAE_AutoTS_w_365': 1130.8769230769228,
  'MSE_AutoTS_w_365': 1423005.0563461534},
 {'Profile': '13ee2c9dc0ad

## 300-365

In [ ]:
# Convert date column to datetime
between_365_300_data["date"] = pd.to_datetime(between_365_300_data["date"])

# Extract year and week number
between_365_300_data["year"] = between_365_300_data["date"].dt.year
between_365_300_data["week"] = between_365_300_data["date"].dt.isocalendar().week  # ISO week number

# Aggregate: Keep last follower count per week for each profile
weekly_df = (
    between_365_300_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "week"])
    .last()
    .reset_index()
)

# Convert back to a proper date format (set to Monday of the given week)
weekly_df["date"] = pd.to_datetime(weekly_df["year"].astype(str) + weekly_df["week"].astype(str) + '1', format='%G%V%w')

# Drop unnecessary columns
weekly_df = weekly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Extract unique IDs from the DataFrame
unique_ids = weekly_df['profile_id'].unique()

# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Create a complete date range for each profile
full_date_range = pd.date_range(start="2023-01-02", end="2024-12-30", freq="W-MON")
full_index = pd.MultiIndex.from_product([selected_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame
weekly_df = weekly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Filter the DataFrame to include only the selected IDs
weekly_df = weekly_df[weekly_df['profile_id'].isin(selected_ids)]

weekly_df.fillna(method='ffill', inplace=True)
weekly_df.fillna(method='bfill', inplace=True)
weekly_df.fillna(0, inplace=True)

# Split the data into training and testing sets
train_df = weekly_df[weekly_df['date'] < '2024-07-01']
test_df = weekly_df[weekly_df['date'] >= '2024-07-01']

# Pivot and prepare for AutoTS
def prepare_for_autots(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')  # Forward-fill missing values
    df.index = df.index.to_period('W').to_timestamp('W')  # Convert index to week-end format
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autots(train_df)
test_df = prepare_for_autots(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

evaluation_results_w_300 = []

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='W',  # Changed frequency to weekly
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_w_300.append({'Profile': profile_id, 'MAE_AutoTS_w_300': mae_autots, 'MSE_AutoTS_w_300': mse_autots})

# Convert to DataFrame
evaluation_df_w_300 = pd.DataFrame(evaluation_results_w_300)

In [ ]:
evaluation_results_w_300

## 90-299

In [76]:
# get the ids of the accounts that have 50 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 50 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [ ]:
# Convert date column to datetime
between_299_90_data["date"] = pd.to_datetime(between_299_90_data["date"])

# Extract year and week number
between_299_90_data["year"] = between_299_90_data["date"].dt.year
between_299_90_data["week"] = between_299_90_data["date"].dt.isocalendar().week  # ISO week number

# Aggregate: Keep last follower count per week for each profile
weekly_df = (
    between_299_90_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "week"])
    .last()
    .reset_index()
)

# Convert back to a proper date format (set to Monday of the given week)
weekly_df["date"] = pd.to_datetime(weekly_df["year"].astype(str) + weekly_df["week"].astype(str) + '1', format='%G%V%w')

# Drop unnecessary columns
weekly_df = weekly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Extract unique IDs from the DataFrame
unique_ids = weekly_df['profile_id'].unique()

# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Create a complete date range for each profile
full_date_range = pd.date_range(start="2023-01-02", end="2024-12-30", freq="W-MON")
full_index = pd.MultiIndex.from_product([selected_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame
weekly_df = weekly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Filter the DataFrame to include only the selected IDs
weekly_df = weekly_df[weekly_df['profile_id'].isin(selected_ids)]

weekly_df.fillna(method='ffill', inplace=True)
weekly_df.fillna(method='bfill', inplace=True)
weekly_df.fillna(0, inplace=True)

# Split the data into training and testing sets
train_df = weekly_df[weekly_df['date'] < '2024-07-01']
test_df = weekly_df[weekly_df['date'] >= '2024-07-01']

# Pivot and prepare for AutoTS
def prepare_for_autots(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')  # Forward-fill missing values
    df.index = df.index.to_period('W').to_timestamp('W')  # Convert index to week-end format
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autots(train_df)
test_df = prepare_for_autots(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

evaluation_results_w_90 = []

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='W',  # Changed frequency to weekly
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_w_90.append({'Profile': profile_id, 'MAE_AutoTS_w_50': mae_autots, 'MSE_AutoTS_w_50': mse_autots})

# Convert to DataFrame
evaluation_df_w_90 = pd.DataFrame(evaluation_results_w_90)

In [ ]:
evaluation_results_w_90

In [80]:
evaluation_df_w_90

,Profile,MAE_AutoTS_w_50,MSE_AutoTS_w_50
0,00ed31934cb3ec073edcc77b23fd93bada5ee0047ddd31...,6.542378e+04,4.280460e+09
1,0967bebbf092787a3d3400ce1c1a012d88cbc1af75b7f7...,6.883733e+03,1.115122e+08
2,0d40472f6bca0211ad17770e9990344d16cf05b1f0fcd5...,3.341385e+03,1.549067e+07
3,103a0b7815fdec7b7920309d62b5bfb1d36bed9b909c68...,8.399406e+04,7.055003e+09
4,14c9357c4e4621e88d078764f978b2d0afed460175dc69...,0.000000e+00,0.000000e+00
5,1672870e984cf7cbc3d10e576d66acf1b25b5a029b2282...,1.177063e+06,2.002506e+12
6,167d0013957e4cbf435c12d3fd63e3001392dc37d7b90f...,1.515825e-13,3.446586e-26
7,1aa839d0b39346e243f7e5499b1afcc020ebf829307dda...,6.230000e+01,3.881290e+03
8,1ea02215d567605cc3ae010a3783a1b3c4a98e283fb335...,4.138819e+04,1.713311e+09
9,1ec8fb36c578c721e00d4143c6d2c2f66bbffb4d7fdb58...,5.772400e+03,3.332060e+07
